In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
metadata_path = './skin-cancer-mnist-ham10000/HAM10000_metadata.csv'
metadat = pd.read_csv(metadata_path)
num_samples = len(metadat)
metadat = metadat.sample(frac=1,random_state=12).reset_index(drop=True) #shuffling data

metadat.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0003559,ISIC_0025810,bkl,consensus,75.0,male,face
1,HAM_0004240,ISIC_0032517,mel,histo,50.0,male,back
2,HAM_0002610,ISIC_0026876,vasc,consensus,0.0,female,abdomen
3,HAM_0003229,ISIC_0031140,bcc,histo,60.0,male,chest
4,HAM_0005963,ISIC_0028129,nv,follow_up,45.0,male,foot


In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        directory=r'./train',
        target_size=(224, 224),
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        seed=42)

val_generator = val_datagen.flow_from_directory(
        directory=r'./val',
        target_size=(224, 224),
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        seed=12)

test_generator = test_datagen.flow_from_directory(
        directory=r'./test',
        target_size=(224, 224),
        batch_size=1,
        class_mode=None,
        shuffle=False,
        seed=12)

Found 6408 images belonging to 7 classes.
Found 1602 images belonging to 7 classes.
Found 2003 images belonging to 7 classes.


In [4]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras.optimizers import SGD

# create the base pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False)

# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

# add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(7, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=["accuracy"])

for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_1
5 max_pooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 add_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 add_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 add_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 add_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 add_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch2a
62 activatio

In [6]:
# train the model on the new data for a few epochs
history = model.fit_generator(train_generator,
        steps_per_epoch=len(train_generator)//32,
        epochs=50,
        validation_data=val_generator,
        validation_steps=len(val_generator)//32)

Epoch 1/50
6/6 [==============================] - 10s 2s/step - loss: 2.5572 - acc: 0.1250 - val_loss: 2.2838 - val_acc: 0.1250
Epoch 2/50
6/6 [==============================] - 7s 1s/step - loss: 2.1420 - acc: 0.2240 - val_loss: 1.9651 - val_acc: 0.1250
Epoch 3/50
6/6 [==============================] - 7s 1s/step - loss: 1.5092 - acc: 0.4531 - val_loss: 1.6775 - val_acc: 0.1250
Epoch 4/50
6/6 [==============================] - 7s 1s/step - loss: 1.3910 - acc: 0.6250 - val_loss: 1.5047 - val_acc: 0.6250
Epoch 5/50
6/6 [==============================] - 8s 1s/step - loss: 1.4001 - acc: 0.6615 - val_loss: 1.4303 - val_acc: 0.6250
Epoch 6/50
6/6 [==============================] - 7s 1s/step - loss: 1.3694 - acc: 0.6927 - val_loss: 1.4152 - val_acc: 0.6250
Epoch 7/50
6/6 [==============================] - 9s 1s/step - loss: 1.5748 - acc: 0.6094 - val_loss: 1.4283 - val_acc: 0.6250
Epoch 8/50
6/6 [==============================] - 8s 1s/step - loss: 1.5713 - acc: 0.6198 - val_loss: 1.4598 -

In [7]:
model.save('./models/resnet_epoch50.h5')

In [22]:
model.evaluate_generator(generator=val_generator,
steps=STEP_SIZE_VALID)

2

In [21]:
len(y_pred[0])

7